In [ ]:
import transformers
import torch
import pandas as pd
from huggingface_hub import interpreter_login
import re

In [ ]:
# Authenticate with Huggingface token
#!git config --global credential.helper store
interpreter_login()

In [ ]:
# Load evaluation data, 1000 samples
data = pd.read_csv("", usecols = ["text", "label"])

In [ ]:
# Prompt
sentiment_v3_prompt = """
Du er en gennemsnitlig dansk nyhedsforbruger. Du får en overskrift og underoverskrift på en artikel, og skal tildele den en kategori svarende til det sentiment den fremkalder. 
Her er nogle generelle principper, du skal følge:
Du skal bruge din viden om det danske samfund og annotere artiklen som du forestiller dig, at den gennemsnitlige dansker ville gøre det.
Undgå at lade dig påvirke af personlige holdninger og bias. Nogle artikler udtrykker holdninger som du måske er uenig med, men det må ikke påvirke vurderingen af artiklens sentiment.
Du skal tildele artiklen dens mest dominerende sentiment, hvis den vurderes at indeholde flere sentiments.
Der findes både explicit og implicit sentiment. Explicit sentiment afspejler ofte nogens indre tilstand (f.eks. tro, holdninger, tanker, følelser osv.). Implicit sentiment afspejler derimod ofte en kendsgerning, der fører til et positivt eller negativt sentiment omkring et emne (f.eks. en person eller begivenhed). Artikler fremkalder ofte implicit sentiment ved at fremhæve gode eller dårlige begivenheder. F.eks. er begivenheder som fødsler, at blive gift eller forfremmet gode begivenheder, mens f.eks. død og sygdom er dårlige begivenheder.
Du skal bruge din generelle viden om følelsesladede udtryk på dansk. Vær opmærksom på, at sentiment kan forekomme ved ord, der normalt ikke betragtes som følelsesladede, f.eks. familie, løn og ansættelse. Nogle begivenheder er også forbundet med sentiment, f.eks. 1. verdenskrig eller Covid-19.
Kategorier: ”Positiv”: Fremkalder en overordnet positiv sentiment. Stemninger som optimisme, tilfredshed og selvsikkerhed betragtes som positive. ”Negativ”: Fremkalder en overordnet negativ sentiment. Stemninger som vrede, skuffelse og tristhed betragtes som negative. ”Neutral”: Fremkalder hverken en positiv eller negativ sentiment. Enten ingen sentiment eller tvetydig sentiment. 
Giv et præcist svar i json: {{sentiment: ”kategori”}}.
"""

In [ ]:
# Load model pipeline
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct" 

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto"
)

In [ ]:
# Function to run Llama with few-shot prompts
def fewshot_sentiment_annotation(text, prompt, fewshot_dataset):
    # Sample few-shot examples from dataset without current article
    fewshot_dataset = fewshot_dataset[fewshot_dataset["text"] != text]
    fewshot_examples = fewshot_dataset.sample(3).to_dict(orient="records")
    # Create few-shot part of prompt
    fewshot_examples = "\n".join([f"Artikel: {example["text"]}. Artiklen fremkalder dette sentiment: {example["label"]}." for example in fewshot_examples])
    
    messages = [
    {"role": "system", "content": f"{prompt}. Her er tre eksempler på artikler og deres sentiment: {fewshot_examples}"},
    {"role": "user", "content": f"Artikel: {text}. Artiklen fremkalder dette sentiment: "},
    ]

    outputs = pipeline(
        messages,
        max_new_tokens=256,
    )
    
    # Return the generated content
    return outputs[0]["generated_text"][-1]["content"]

In [ ]:
# Apply few-shot
data["llm_annotation"] = data["text"].apply(lambda text: fewshot_sentiment_annotation(text, prompt=sentiment_v3_prompt, fewshot_dataset=data))

In [ ]:
# save df to csv
data.to_csv("")